# 4.1　欠測データへの対処
- ほとんどの計算ツールは欠測値に対処できないか、欠測値を無視した場合に予期せぬ結果を生み出す。
- そのため、分析をすすめる前に、欠測値に対処することがきわめて重要となる

In [1]:
#サンプルデータで例示
import pandas as pd
from io import StringIO #ハードディスク上の通常のCSVと同じように、変数csv_dataに代入した文字列を扱うため

#サンプルデータの作成
csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''
# python2.7を使用している場合は文字列をunicodeに変化する必要がある
# csv_data = unicode(csv_data)
#サンプルデータを読み込む
csv_data = unicode(csv_data)
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [2]:
#各特徴量の欠測値をカウント
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

##### コラム
- scikit-learnはNumPyの配列に対応するように開発されている
- DataFrameオブジェクトはNumpyの配列であり、value属性を使っていつでもアクセスできる。

# 4.1.1 欠測値をもつサンプル／特徴量を取り除く
- 欠測地に対処する最も簡単な方法の１つは、該当する列または行をデータセットから削除することである。

In [3]:
#欠測値を含む行を削除
df.dropna()

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [4]:
#欠測値を含む列を削除
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [5]:
#すべての列がNaNである列だけを削除
df.dropna(how='all')
#非NaN値が4つ未満の行を削除
df.dropna(thresh=4)
#特定の列にNaNが含まれている行だけを削除
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


- サンプルを削除しすぎてしまうと、解析の信頼性が失われることがあるので注意
- 特徴量の列を削除しすぎてしまうと、分類器がクラスを識別するのに必要な、有益な情報が失われる恐れがある。

# 4.1.2 欠測値を補完する
- 行を削除したり、列を削除したりすると、貴重なデータを多く失いかねないため、そうした方法はたいてい現実的ではない。
- 補間法を用いて欠測値を推測する。
- 最も一般的な補間法の１つは、平均値補完である。平均値補完は、単に欠測値を特徴量の列全体の平均値と置き換える

In [6]:
from sklearn.preprocessing import Imputer
#欠測値補完のインスタンスを生成
imr = Imputer(missing_values='NaN',strategy='mean',axis=0)#strategyには他に、median,most_frequentがある
# データを適合
imr = imr.fit(df)
#補完を実行
imputed_data = imr.transform(df.values)
imputed_data

array([[  1. ,   2. ,   3. ,   4. ],
       [  5. ,   6. ,   7.5,   8. ],
       [ 10. ,  11. ,  12. ,   6. ]])

# 4.1.3 scikit-learnの推定器API
- Imputerクラスはskleranの変換器クラスに属している。変換器クラスはデータの変換に使用される
- 変換器には基本的なメソッドとして、fitとtransformの２つがある。
- fitメソッドはトレーニングセットからパラメータを学習するために利用される。
- transformメソッドは学習したパラメータに基づいてデータを変換するために使用される

# 4.2 カテゴリデータの処理
- カテゴリデータに関しては、名義特徴量と順序特徴量を区別する必要がある。

In [20]:
import pandas as pd
#サンプルデータを生成（Tシャツの色・サイズ・価格・クラスラベル）
df = pd.DataFrame([
        ['green','M',10.1,'class1'],
        ['red','L',13.5,'class2'],
        ['blue','XL',15.3,'class1']])

#列名を指定
df.columns = ['color','size','price','classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


- 本書で説明する分類用の学習アルゴリズムでは、クラスラベルに順序の情報を使用しない。

# 4.2.1 順序特徴量のマッピング

In [21]:
#Tシャツのサイズと整数を対応させるディクショナリを生成
size_mapping = {'XL':3,'L':2,'M':1}
#Tシャツのサイズを整数に変換
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


# クラスラベルのエンコーディング
- 多くの機械学習ライブラリは、クラスラベルが整数値としてエンコードされていることを要求する。
- 予めクラスラベルを整数の配列として提供するのがよいプラクティスである。

In [22]:
import numpy as np
#クラスラベルと整数を対応させるディクショナリを生成
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [23]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [24]:
#変換されたラベルを元の文字列表現にもどす
#整数からクラスラベルに変換
inv_class_mapping = {v:k for k,v in class_mapping.items()}
#整数からクラスラベルに変換
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [34]:
#sklearnのLabelEncoderを利用する方法もある。
from sklearn.preprocessing import LabelEncoder
#ラベルエンコーダのインスタンスを生成
class_le = LabelEncoder()
#クラスラベルから整数に変換
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0])

In [28]:
#クラスラベルを文字列に戻す
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

# 名義特徴量でのone-hotエンコーディング
- 名義特徴量には、順序の概念はないが、前節の方法をcolor列に適用すると学習の際に順序があるものとして処理されてしまう
- この問題を解する方法は、one-hotエンコーディングという手法である。
- 名義特徴量の列の一意な値ごとにダミー特徴量を新たに作成するという発想に基づいている。

In [36]:
#Tシャツの色、サイズ、価格を抽出
X = df[['color','size','price']].values
color_le = LabelEncoder()
X[:,0] = color_le.fit_transform(X[:,0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

In [39]:
from sklearn.preprocessing import OneHotEncoder
#one-hotエンコーダの生成
#categorical_featureで列を指定
ohe = OneHotEncoder(categorical_features=[0])
#one-hotエンコーディングを実行
ohe.fit_transform(X).toarray()

array([[  0. ,   1. ,   0. ,   1. ,  10.1],
       [  0. ,   0. ,   1. ,   2. ,  13.5],
       [  1. ,   0. ,   0. ,   3. ,  15.3]])

In [40]:
# one-hotエンコーディングを実行
pd.get_dummies(df[['price','color','size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0.0,1.0,0.0
1,13.5,2,0.0,0.0,1.0
2,15.3,3,1.0,0.0,0.0


- pandasで実装されているget_dummiesのほうが明らかに便利